# Practice PS06: Recommendations engines (interactions-based)

For this assignment we will build and apply an item-based and model-based collaborative filtering recommenders for movies. 

Author: <font color="blue">José M. Pérez Clar</font>

E-mail: <font color="blue">joseperezclar@gmail.com</font>

Date: <font color="blue">08/11/2023</font>

# 1. The Movies dataset

# 1.1. Load the input files

In [1]:
# Leave this code as-is

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from math import*
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# Leave this code as-is

FILENAME_MOVIES = "movies-2000s.csv"
FILENAME_RATINGS = "ratings-2000s.csv"
FILENAME_TAGS = "tags-2000s.csv"

In [3]:
# Leave this code as-is

movies = pd.read_csv(FILENAME_MOVIES, 
                    sep=',', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
display(movies.head(5))

ratings_raw = pd.read_csv(FILENAME_RATINGS, 
                    sep=',', 
                    encoding='latin-1',
                    engine='python',
                    names=['user_id', 'movie_id', 'rating'])
display(ratings_raw.head(5))

,movie_id,title,genres
0,2769,"Yards, The (2000)",Crime|Drama
1,3177,Next Friday (2000),Comedy
2,3190,Supernova (2000),Adventure|Sci-Fi|Thriller
3,3225,Down to You (2000),Comedy|Romance
4,3228,Wirey Spindell (2000),Comedy


,user_id,movie_id,rating
0,4,1,3.0
1,4,260,3.5
2,4,296,4.0
3,4,541,4.5
4,4,589,4.0


# 1.2. Merge the data into a single dataframe

In [4]:
ratings = pd.merge(movies, ratings_raw, how='inner', on='movie_id')
display(ratings.head(5))

,movie_id,title,genres,user_id,rating
0,2769,"Yards, The (2000)",Crime|Drama,1115,4.0
1,2769,"Yards, The (2000)",Crime|Drama,1209,2.0
2,2769,"Yards, The (2000)",Crime|Drama,2004,3.0
3,2769,"Yards, The (2000)",Crime|Drama,2502,4.0
4,2769,"Yards, The (2000)",Crime|Drama,2827,4.0


In [5]:
def find_movies(keyword, movies):
    for index, movie in movies.iterrows():
        if keyword in movie['title']:
            print("movie id:" , movie['movie_id'] , "title:", movie['title'])

In [6]:
# LEAVE AS-IS

# For testing, this should print:
# movie_id:  4993, title: Lord of the Rings: The Fellowship of the Ring, The (2001)
# movie_id:  5952, title: Lord of the Rings: The Two Towers, The (2002)
# movie_id:  7153, title: Lord of the Rings: The Return of the King, The (2003)
find_movies("Lord of the Rings", movies)

movie id: 4993 title: Lord of the Rings: The Fellowship of the Ring, The (2001)
movie id: 5952 title: Lord of the Rings: The Two Towers, The (2002)
movie id: 7153 title: Lord of the Rings: The Return of the King, The (2003)


In [7]:
# LEAVE AS-IS

def get_title(movie_id, movies):
    return movies[movies['movie_id'] == movie_id].title.iloc[0]

In [8]:
# LEAVE AS-IS

# For testing, should print "Lord of the Rings: The Return of the King, The (2003)")
print(get_title(7153, movies))

Lord of the Rings: The Return of the King, The (2003)


## 1.3. Count unique registers

In [9]:
num_users = len(ratings['user_id'].unique())
num_movies_rated = len(ratings['movie_id'].unique())
num_movies = len(movies['movie_id'].unique())

print("Number of users who have rated a movie:", num_users)
print("Number of movies that have been rated:", num_movies_rated)
print("Total number of movies:", num_movies)


Number of users who have rated a movie: 12676
Number of movies that have been rated: 2049
Total number of movies: 33168


# 2. Item-based Collaborative Filtering

## 2.1. Data pre-processing

In [10]:
rated_movies = ratings.drop('genres', axis=1)
display(rated_movies.head(10))

,movie_id,title,user_id,rating
0,2769,"Yards, The (2000)",1115,4.0
1,2769,"Yards, The (2000)",1209,2.0
2,2769,"Yards, The (2000)",2004,3.0
3,2769,"Yards, The (2000)",2502,4.0
4,2769,"Yards, The (2000)",2827,4.0
5,2769,"Yards, The (2000)",6629,1.0
6,2769,"Yards, The (2000)",12435,4.0
7,2769,"Yards, The (2000)",13873,3.0
8,2769,"Yards, The (2000)",14799,3.0
9,2769,"Yards, The (2000)",15691,2.5


In [11]:
# Initialize ratings_summary to be only the movie_id and title of all movies in rated_movies
ratings_summary = rated_movies[['movie_id', 'title']].groupby('movie_id').first().copy()

ratings_mean = rated_movies.groupby('movie_id')['rating'].mean()
ratings_count = rated_movies.groupby('movie_id')['rating'].count()

ratings_summary['ratings_mean'] = ratings_mean
ratings_summary['ratings_count'] = ratings_count

display(ratings_summary.head(5))

,title,ratings_mean,ratings_count
movie_id,,,
2769,"Yards, The (2000)",3.122549,102
3177,Next Friday (2000),2.824000,125
3190,Supernova (2000),2.395683,139
3225,Down to You (2000),2.577273,110
3228,Wirey Spindell (2000),2.500000,2


In [12]:
popular_movies = ratings_summary[ratings_summary['ratings_count'] >= 2500]
top_rated_movies = popular_movies.sort_values(by='ratings_mean', ascending=False)

print("Top 5 Highest Rated Movies with at least 2500 ratings:")
print(top_rated_movies.head(5)[['title', 'ratings_mean', 'ratings_count']])

Top 5 Highest Rated Movies with at least 2500 ratings:
                                                      title  ratings_mean  \
movie_id                                                                    
4226                                         Memento (2000)      4.158512   
4973      Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...      4.097234   
4993      Lord of the Rings: The Fellowship of the Ring,...      4.092530   
7153      Lord of the Rings: The Return of the King, The...      4.083960   
5952          Lord of the Rings: The Two Towers, The (2002)      4.083869   

          ratings_count  
movie_id                 
4226               4476  
4973               3687  
4993               5944  
7153               5449  
5952               5449  


In [13]:
popular_movies_ = ratings_summary[ratings_summary['ratings_count'] >= 3]
top_rated_movies_ = popular_movies_.sort_values(by='ratings_mean', ascending=False)

print("Top 5 Highest Rated Movies with at least 2500 ratings:")
print(top_rated_movies_.head(5)[['title', 'ratings_mean', 'ratings_count']])

Top 5 Highest Rated Movies with at least 2500 ratings:
                                         title  ratings_mean  ratings_count
movie_id                                                                   
5082                 Rumor of Angels, A (2000)      4.666667              6
27764                              2LDK (2003)      4.500000              3
31954     Beautiful City (Shah-re ziba) (2004)      4.400000              5
5224                           Promises (2001)      4.388889             18
6775                      Life and Debt (2001)      4.333333              3


There is not any movie in common for both results. This is understandable since when the number of ratings is very low it's easier to average a high rating. Nonetheless, when the number of ratings is very high, following the CLT theorem, the ratings are normalised. Basically, what this means is that the higher the number of ratings, the more probabilites  of receiving a low rating that will bring your avg down.

## 2.2. Compute the user-movie matrix

In [14]:
user_movie = rated_movies.pivot_table(index='user_id', columns='movie_id', values='rating')
print(user_movie.head(5))

movie_id  2769   3177   3190   3225   3228   3239   3273   3275   3276   \
user_id                                                                   
4           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
62          NaN    NaN    NaN    NaN    NaN    NaN    NaN    4.5    NaN   
63          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
95          NaN    NaN    NaN    NaN    NaN    NaN    NaN    3.5    NaN   

movie_id  3279   ...  33138  33145  33148  33150  33152  33154  33158  33162  \
user_id          ...                                                           
4           NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33          NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
62          NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
63          NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    N

The reason why the "user_movie" matrix contains NaN" values is because not all users have rated every movie. In a recommender system users only provide ratings for a number of movies and each user has their own specific set of rated movies. Consequently the matrix appears very sparse with entries.

The charcteristic is hence known as "sparsity." Sparsity occurs when the number of items (movies in this case) is much larger than the number of ratings provided by users. 

# 2.3. Explore some correlations in the user-movie matrix

In [15]:
print(find_movies("Lord of the Rings" , movies))
print(find_movies("Finding Nemo (2003)" , movies))
print(find_movies("Talk to Her (Hable con Ella) (2002)" , movies))

movie id: 4993 title: Lord of the Rings: The Fellowship of the Ring, The (2001)
movie id: 5952 title: Lord of the Rings: The Two Towers, The (2002)
movie id: 7153 title: Lord of the Rings: The Return of the King, The (2003)
None
movie id: 6377 title: Finding Nemo (2003)
None
movie id: 5878 title: Talk to Her (Hable con Ella) (2002)
None


In [16]:
id_pivot = 4993
id_m1 = 6377
id_m2 = 5878

s1 = user_movie[id_pivot].dropna()
s2 = user_movie[id_m1].dropna()
s3 = user_movie[id_m2].dropna()

ratings3 = pd.concat([s1, s2, s3], axis=1)

ratings3 = ratings3.dropna()

print(ratings3.head(10))

         4993  6377  5878
user_id                  
859       3.0   4.0   5.0
1229      4.0   4.0   4.5
1281      3.0   2.5   3.0
1722      5.0   4.5   4.0
2004      4.5   3.0   3.5
4590      4.0   4.0   2.0
5052      2.0   4.0   4.0
5144      5.0   5.0   5.0
6497      3.5   3.5   3.5
8369      3.0   4.0   4.5


In [17]:
corr_1_2 = ratings3[id_pivot].corr(ratings3[id_m1])
corr_1_3 = ratings3[id_pivot].corr(ratings3[id_m2])
corr_2_3 = ratings3[id_m1].corr(ratings3[id_m2])

print("Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Finding Nemo (2003)'", corr_1_2)
print("Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Talk to Her (Hable con Ella) (2002)':", corr_1_3)
print("Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)':", corr_2_3)

Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Finding Nemo (2003)' 0.3840549071566769
Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Talk to Her (Hable con Ella) (2002)': 0.16240502267155427
Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': 0.2042645045941217


There is a correlation of (0.38), between the movie 'Lord of the Rings; The Fellowship of the Ring, The (2001)' and 'Finding Nemo (2003)'. Moreover, there is a positive correlation (0.16) between 'Lord of the Rings; The Fellowship of the Ring, The (2001)' and 'Talk to Her (Hable con Ella) (2002)'. Similarly there is also a positive correlation (0.20) between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)'. 

What this numbers mean is that, to different extents, users who rated one of these movies tend to rate the other one similarly (for each pair). It's important to remember that the correlation between two elements goes from [-1,1], so even though not for all pairs this correlation is super strong, it is definetely there. 

In [18]:
pivot_movie_ratings = user_movie[id_pivot]

similar_to_pivot = user_movie.corrwith(pivot_movie_ratings)

similar_to_pivot = similar_to_pivot.dropna()

print(similar_to_pivot)

C:\Users\josep\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\josep\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


movie_id
2769    -0.127515
3177     0.093221
3190     0.041206
3225     0.126600
3239     0.338378
           ...   
33154    0.318255
33158    0.228214
33162    0.285377
33164    0.037130
33166    0.197344
Length: 1868, dtype: float64


In [19]:
corr_with_pivot = pd.DataFrame(similar_to_pivot, columns=['corr'])
corr_with_pivot['title'] = ratings_summary['title']
corr_with_pivot['ratings_mean'] = ratings_summary['ratings_mean']
corr_with_pivot['ratings_count'] = ratings_summary['ratings_count']

corr_with_pivot = corr_with_pivot[corr_with_pivot['ratings_count'] > 500]

top_10_similar_movies = corr_with_pivot.sort_values('corr', ascending=False).head(10)
display(top_10_similar_movies)

,corr,title,ratings_mean,ratings_count
movie_id,,,,
4993,1.000000,"Lord of the Rings: The Fellowship of the Ring,...",4.092530,5944
5952,0.892103,"Lord of the Rings: The Two Towers, The (2002)",4.083869,5449
7153,0.892073,"Lord of the Rings: The Return of the King, The...",4.083960,5449
6539,0.377599,Pirates of the Caribbean: The Curse of the Bla...,3.779241,3950
8368,0.340934,Harry Potter and the Prisoner of Azkaban (2004),3.809971,2397
3578,0.337667,Gladiator (2000),3.951050,4811
3793,0.329686,X-Men (2000),3.556436,3535
4896,0.319180,Harry Potter and the Sorcerer's Stone (a.k.a. ...,3.678509,2843
3624,0.307471,Shanghai Noon (2000),3.297443,1017


The way movies are chosen changes when we change the condition on ratings_count. Based on what occurs when we set the condition to a higher and a lower value, we can split the answer as follows:

Higher Value: The corr_with_pivot dataframe will only contain movies with a high number of ratings, therefore the popular and extensively viewed films will probably be included in the resulting dataframe. This strategy might disregard smaller films in favor of focusing more on well-known ones.

Lower Value: The corr_with_pivot dataframe will also include movies with fewer ratings, which means that the final dataframe will be made up also of films that are not as well-known or don't have as large of a public. The suggestions that are produced are more adapted to particular preferences but might not include highly rated super popular movies.


# 2.4. Implement the item-based recommendations

In [20]:
item_similarity = user_movie.corr()
display(item_similarity.head(10))

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
movie_id,,,,,,,,,,,,,,,,,,,,,
2769,1.000000,0.115068,0.033721,-0.232268,NaN,-0.500000,0.197011,0.199514,0.250873,NaN,...,0.379980,0.878310,NaN,NaN,NaN,0.248126,1.806095e-01,-0.085570,-0.408248,0.105671
3177,0.115068,1.000000,0.303820,0.559533,NaN,NaN,0.331191,0.167918,1.000000,NaN,...,0.546119,0.735767,-1.0,NaN,NaN,-0.221382,3.174747e-01,0.014735,0.661989,0.185654
3190,0.033721,0.303820,1.000000,0.636361,NaN,-0.014315,0.146042,0.394293,-0.290397,NaN,...,0.246183,0.632026,NaN,NaN,NaN,0.378181,1.709261e-01,0.022444,-0.073360,-0.054114
3225,-0.232268,0.559533,0.636361,1.000000,NaN,0.578414,0.347716,0.263671,-0.250313,NaN,...,-0.300376,0.318377,NaN,NaN,NaN,0.480173,7.503063e-01,0.536828,0.753141,0.098748
3228,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3239,-0.500000,NaN,-0.014315,0.578414,NaN,1.000000,0.180846,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,1.000000,0.636285,0.888200
3273,0.197011,0.331191,0.146042,0.347716,NaN,0.180846,1.000000,0.105735,0.154371,NaN,...,0.006774,0.409968,1.0,NaN,NaN,0.088405,7.516779e-02,0.143492,0.466705,0.084202
3275,0.199514,0.167918,0.394293,0.263671,NaN,1.000000,0.105735,1.000000,0.485071,NaN,...,-0.011426,0.279624,NaN,NaN,NaN,0.075827,2.994603e-01,0.187713,0.285584,0.225317
3276,0.250873,1.000000,-0.290397,-0.250313,NaN,NaN,0.154371,0.485071,1.000000,NaN,...,NaN,0.292770,NaN,NaN,NaN,0.000000,-6.885311e-17,-0.455530,0.500000,-0.138013


In [21]:
item_similarity_min_ratings = user_movie.corr(min_periods=100)
display(item_similarity_min_ratings.head(5))

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
movie_id,,,,,,,,,,,,,,,,,,,,,
2769,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3177,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3190,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
user_id_super = user_movie[(user_movie[5349]>4.5) & (user_movie[3793]>4.5) & (user_movie[6534]>4.5)].index
user_id_drama = user_movie[(user_movie[6870]>4.5) & (user_movie[5995]>4.5) & (user_movie[3555]>4.5)].index

print("user_id_super:", user_id_super)
print("user_id_drama:", user_id_drama)

user_id_super: Int64Index([127342], dtype='int64', name='user_id')
user_id_drama: Int64Index([34336, 68770, 147353], dtype='int64', name='user_id')


Then, based on the results above, we can make the following decisions.

In [23]:
user_id_super= 127342
user_id_drama = 34336

In [24]:
# Leave this code as-is

# Gets a list of watched movies for a user_id
def get_watched_movies(user_id, user_movie):
    return list(user_movie.loc[user_id].dropna().sort_values(ascending=False).index)
    
# Gets the rating a user_id has given to a movie_id
def get_rating(user_id, movie_id, user_movie):
    return user_movie[movie_id][user_id]

# Print watched movies
def print_watched_movies(user_id, user_movie, movies):
    for movie_id in get_watched_movies(user_id, user_movie):
        print("%d %.1f %s " %
          (movie_id, get_rating(user_id, movie_id, user_movie), get_title(movie_id, movies)))


In [25]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_super, user_movie, movies)

5502 5.0 Signs (2002) 
5445 5.0 Minority Report (2002) 
6156 5.0 Shanghai Knights (2003) 
5952 5.0 Lord of the Rings: The Two Towers, The (2002) 
5944 5.0 Star Trek: Nemesis (2002) 
5816 5.0 Harry Potter and the Chamber of Secrets (2002) 
5618 5.0 Spirited Away (Sen to Chihiro no kamikakushi) (2001) 
5524 5.0 Blue Crush (2002) 
5480 5.0 Stuart Little 2 (2002) 
5459 5.0 Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (2002) 
5420 5.0 Windtalkers (2002) 
4388 5.0 Scary Movie 2 (2001) 
5389 5.0 Spirit: Stallion of the Cimarron (2002) 
5349 5.0 Spider-Man (2002) 
5218 5.0 Ice Age (2002) 
5064 5.0 The Count of Monte Cristo (2002) 
4993 5.0 Lord of the Rings: The Fellowship of the Ring, The (2001) 
4973 5.0 Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001) 
4896 5.0 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) 
4886 5.0 Monsters, Inc. (2001) 
6186 5.0 Gods and Generals (2003) 
6333 5.0 X2: X-Men United (2003) 
6377 5.0 Finding Nemo (2003) 
6

In [26]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_drama, user_movie, movies)

3967 5.0 Billy Elliot (2000) 
4014 5.0 Chocolat (2000) 
4034 5.0 Traffic (2000) 
5995 5.0 Pianist, The (2002) 
7147 5.0 Big Fish (2003) 
4995 5.0 Beautiful Mind, A (2001) 
3555 5.0 U-571 (2000) 
6870 5.0 Mystic River (2003) 
5991 5.0 Chicago (2002) 
8464 5.0 Super Size Me (2004) 
5669 5.0 Bowling for Columbine (2002) 
8622 5.0 Fahrenheit 9/11 (2004) 
30707 5.0 Million Dollar Baby (2004) 
6953 4.5 21 Grams (2003) 
5015 4.5 Monster's Ball (2001) 
5464 4.5 Road to Perdition (2002) 
3510 4.5 Frequency (2000) 
5989 4.5 Catch Me If You Can (2002) 
4022 4.0 Cast Away (2000) 
5010 4.0 Black Hawk Down (2001) 
5299 4.0 My Big Fat Greek Wedding (2002) 
3897 4.0 Almost Famous (2000) 
3755 4.0 Perfect Storm, The (2000) 
4308 4.0 Moulin Rouge (2001) 
4447 3.5 Legally Blonde (2001) 
4246 3.5 Bridget Jones's Diary (2001) 
4975 3.5 Vanilla Sky (2001) 
4019 3.5 Finding Forrester (2000) 
5377 3.5 About a Boy (2002) 
3948 3.5 Meet the Parents (2000) 
5956 3.0 Gangs of New York (2002) 
6281 3.0 Phone Booth

In [27]:
def get_movies_relevance(user_id, user_movie, item_similarity_matrix):

    movies_relevance = pd.Series()

    for watched_movie in user_movie.columns:
        rating_given = user_movie.loc[user_id, watched_movie]

        similarities = item_similarity_matrix[watched_movie]

        weighted_similarities = similarities * rating_given

        movies_relevance = movies_relevance.append(weighted_similarities)

    movies_relevance = movies_relevance.groupby(movies_relevance.index).sum()

    movies_relevance_df = pd.DataFrame(movies_relevance, columns=['relevance'])
    movies_relevance_df['movie_id'] = movies_relevance_df.index

    return movies_relevance_df

In [28]:
df1 = get_movies_relevance(user_id_super, user_movie, item_similarity)
df2 = get_movies_relevance(user_id_drama, user_movie, item_similarity)

df1_merged = pd.merge(df1, movies, on='movie_id').sort_values(by='relevance', ascending=False)
df2_merged = pd.merge(df2, movies, on='movie_id').sort_values(by='relevance', ascending=False)

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  movies_relevance = pd.Series()
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

...

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_relevance = movies_relevance.append(weighted_similarities)
C:\Users\josep\AppData\Local\Temp\ipykernel_18548\551271018.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

In [29]:
display(df1_merged.head(10))
display(df2_merged.head(10))

,relevance,movie_id,title,genres
1930,394.160783,31636,"Bunker, The (2001)",Drama|Horror|Mystery|Thriller|War
1662,353.463397,27426,"Accidental Spy, The (Dak miu mai shing) (2001)",Action|Comedy|Thriller
1136,350.723308,6761,Tibet: Cry of the Snow Lion (2002),Documentary
1375,320.251163,7477,Eye See You (D-Tox) (2002),Horror|Thriller
1005,300.482169,6375,Gigantic (A Tale of Two Johns) (2002),Documentary
1088,283.913191,6651,Ash Wednesday (2002),Crime|Drama
1518,282.371780,8835,Paparazzi (2004),Drama|Thriller
562,277.711932,5170,All About the Benjamins (2002),Action|Comedy|Crime
1040,277.051465,6544,"Housekeeper, The (femme de mÃ©nage, Une) (2002)",Comedy|Drama|Romance
1087,274.349794,6648,Chunhyang (2000),Drama|Musical|Romance


,relevance,movie_id,title,genres
1376,160.000000,7521,Mercy (2000),Crime|Mystery|Thriller
351,154.388241,4449,Adanggaman (2000),Drama
1357,146.447098,7443,This So-Called Disaster (2003),Documentary
1930,135.966211,31636,"Bunker, The (2001)",Drama|Horror|Mystery|Thriller|War
1823,133.500000,27835,"Agronomist, The (2003)",Documentary
798,132.174524,5806,Blackboards (TakhtÃ© Siah) (2000),Drama
1098,131.791755,6688,Autumn Spring (BabÃ­ lÃ©to) (2001),Comedy|Drama
986,130.500000,6336,Marooned in Iraq (Gomgashtei dar Aragh) (2002),Drama
233,130.500000,4150,Signs & Wonders (2001),Drama
1040,130.122867,6544,"Housekeeper, The (femme de mÃ©nage, Une) (2002)",Comedy|Drama|Romance


Starting by the superhero fan user suggestions, none of the movies seem to be other superhero movies, which is a bad starting point. Nonetheless, there are a few thrillers and action movies which seem to be more accurate. A good example is "All about the Benjamins", a movie starring Will Smith which has a very similar balance of action and humour compared to Spider-man, for instance. The docummentary recommendations are not very on point. About 6/10 are relevant.

On the other hand, the drama fan user seems to have gotten better results. Most of the movies are classified as drama genre, which a great starting point. Other than that we see the "This So-Called Disaster (2003)" documentary which has been recommended once again and seems to be a not-so-accurate recommendation. About 8/10 recommendations seem relevant.

The fact that there is a not a superhero genre might have hurt the effectiveness of the results of the recommendations for the superhero fan user.

In [30]:
df1_merged.set_index('movie_id', inplace=True)
watched_movies = get_watched_movies(user_id_super, user_movie)
df1_merged.drop(watched_movies, inplace=True)

df2_merged.set_index('movie_id', inplace=True)
watched_movies2 = get_watched_movies(user_id_drama, user_movie)
df2_merged.drop(watched_movies2, inplace=True)

In [31]:
display(df1_merged.head(10))
display(df2_merged.head(10))

,relevance,title,genres
movie_id,,,
31636,394.160783,"Bunker, The (2001)",Drama|Horror|Mystery|Thriller|War
27426,353.463397,"Accidental Spy, The (Dak miu mai shing) (2001)",Action|Comedy|Thriller
7477,320.251163,Eye See You (D-Tox) (2002),Horror|Thriller
6375,300.482169,Gigantic (A Tale of Two Johns) (2002),Documentary
6651,283.913191,Ash Wednesday (2002),Crime|Drama
8835,282.371780,Paparazzi (2004),Drama|Thriller
5170,277.711932,All About the Benjamins (2002),Action|Comedy|Crime
6544,277.051465,"Housekeeper, The (femme de mÃ©nage, Une) (2002)",Comedy|Drama|Romance
6648,274.349794,Chunhyang (2000),Drama|Musical|Romance


,relevance,title,genres
movie_id,,,
7521,160.000000,Mercy (2000),Crime|Mystery|Thriller
4449,154.388241,Adanggaman (2000),Drama
7443,146.447098,This So-Called Disaster (2003),Documentary
31636,135.966211,"Bunker, The (2001)",Drama|Horror|Mystery|Thriller|War
27835,133.500000,"Agronomist, The (2003)",Documentary
5806,132.174524,Blackboards (TakhtÃ© Siah) (2000),Drama
6688,131.791755,Autumn Spring (BabÃ­ lÃ©to) (2001),Comedy|Drama
6336,130.500000,Marooned in Iraq (Gomgashtei dar Aragh) (2002),Drama
4150,130.500000,Signs & Wonders (2001),Drama


In general, even though the recommendations seem relevant looking at the movies, their genres and the plots, they could definetely be improved. The recommendations have barely changed whatsoever after having removed the already watched movies, which is a bad signal. If the recommendations would have been good enough and adapted enough to the likings of each user, we should have seen many more already-watched movies in the first set of recommendations.

Nonetheless, as we could've expected, the relevance scores have gone down, mostly for hero user, after removing already watched movies which were on the list. In the case of the drama user, as none of the recommendations were already watched, the scores of relevance have stayed the same.

In summary, the recommendation engine we designed is a good point to begin with, it does provides recommendations that are meaningul and not completely random, however, it still requires much more development to be as accurate as we expect modern recommendations engines to be.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>